In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yt_dlp
import matplotlib.pyplot as plt
import variables as var
from tqdm import tqdm
import os
import re
import glob

In [ ]:
!export OPENSSL_CONF=/etc/ssl

In [ ]:
# Load the data
DATASET_PATH = '/mnt/d/Thesis/Data/EnTube/Entube.csv'
df = pd.read_csv(DATASET_PATH)
df

In [ ]:
# due to available memory, we will only download 40% of the data
df = df.sample(frac=0.4, random_state=1)
df

In [ ]:
import yt_dlp
import logging
import os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# Setup logging
LOG_FILE = "download_log.txt"
logger = logging.getLogger("yt_dlp")
logger.setLevel(logging.INFO)

file_handler = logging.FileHandler(LOG_FILE)
file_handler.setFormatter(logging.Formatter("%(asctime)s - %(levelname)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S"))
logger.addHandler(file_handler)

def download_video(url, ydl_opts):
    """Download a single video using yt-dlp with logging."""
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        logger.info(f"Successfully downloaded: {url}")
    except Exception as e:
        logger.error(f"Failed to download {url}: {e}")

def download_videos(urls, output_path, max_workers=4):
    """Download multiple videos in parallel, logging all events."""
    ydl_opts = {
        'format': 'bestvideo[height<=480]+bestaudio/best[height<=480]',
        'outtmpl': f"{output_path}/%(id)s",
        'merge_output_format': 'mp4',
        
        'logger': logger,  # Use the logger instance
        'no_warnings': True,   # Suppress warning messages
        'ignoreerrors': True,  # Continue downloading on errors
        
        'writesubtitles': True,  # Download subtitles if available
        'subtitleslangs': ['vi', 'en'],  # Only download Vietnamese and English subtitles
    }

    logger.info(f"Starting download of {len(urls)} videos...")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        list(tqdm(executor.map(lambda url: download_video(url, ydl_opts), urls), total=len(urls), desc="Downloading videos", unit="video"))

    logger.info("Download process completed.")

# Example usage
sample_df = df
video_urls = sample_df['video_link'].tolist()
download_videos(video_urls, var.DOWNLOADED_VIDEO_PATH)

In [ ]:
def read_srt_file(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        subtitles = f.read()
    return subtitles

# Example usage
srt_file_path = f"{var.DOWNLOADED_VIDEO_PATH}/G22G1k3G-kM.vi.vtt"  # Replace with actual path
print(read_srt_file(srt_file_path))